# Indian Financial Transaction Prompt Engineering Tutorial

## Advanced Prompt Engineering Techniques for Indian Financial Services

This comprehensive tutorial demonstrates various prompt engineering techniques specifically tailored for Indian financial transactions, banking operations, and fintech applications. We'll explore everything from basic UPI transaction analysis to complex investment decision-making using Tree of Thoughts methodology.

---

## Table of Contents
1. [Introduction to Indian Financial Context](#1-introduction-to-indian-financial-context)
2. [Core Prompt Engineering Techniques](#2-core-prompt-engineering-techniques)
3. [Tree of Thoughts Methodology](#3-tree-of-thoughts-methodology)
4. [Indian Financial Use Cases](#4-indian-financial-use-cases)
5. [Advanced Techniques & Role-Based Prompting](#5-advanced-techniques--role-based-prompting)
6. [Practical Indian Financial Scenarios](#6-practical-indian-financial-scenarios)
7. [Performance Optimization & Validation](#7-performance-optimization--validation)

---

## Setup and Dependencies

In [ ]:
# Import necessary libraries
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from dotenv import load_dotenv
import json
import datetime
import re
from typing import List, Dict, Any

# Load environment variables
load_dotenv(dotenv_path="/home/bipin/Documents/genai/g25-aug/genai_25_sept/.env")

# Initialize the language model
llm = ChatOpenAI(model="gpt-4o-mini", max_completion_tokens=800, temperature=0.2)

---

## 1. Introduction to Indian Financial Context

### Indian Financial Ecosystem Overview

India's financial landscape is unique with its mix of traditional banking and cutting-edge digital payments:

**Digital Payment Systems:**
- **UPI (Unified Payments Interface)**: Real-time payment system
- **IMPS**: Immediate Payment Service
- **NEFT**: National Electronic Funds Transfer
- **RTGS**: Real Time Gross Settlement

**Regulatory Framework:**
- **RBI**: Reserve Bank of India (Central Bank)
- **KYC**: Know Your Customer compliance
- **AML**: Anti-Money Laundering regulations
- **PMLA**: Prevention of Money Laundering Act

**Common Transaction Types:**
- P2P (Person-to-Person) transfers via UPI
- Bill payments (utilities, mobile, DTH)
- E-commerce transactions
- Investment transactions (SIP, mutual funds)
- Loan EMI payments

### Indian Financial Terminology Quick Reference

In [ ]:
# Indian Financial Terms Dictionary
indian_financial_terms = {
    "UPI": "Unified Payments Interface - Real-time payment system",
    "VPA": "Virtual Payment Address - user@bank format",
    "IFSC": "Indian Financial System Code - 11 character bank identifier",
    "PAN": "Permanent Account Number - 10 character tax identifier",
    "Aadhaar": "12-digit unique identity number",
    "CIBIL": "Credit Information Bureau India Limited - Credit scoring",
    "SIP": "Systematic Investment Plan",
    "EPF": "Employees' Provident Fund",
    "PPF": "Public Provident Fund",
    "GST": "Goods and Services Tax",
    "GSTIN": "GST Identification Number",
    "CTS": "Cheque Truncation System",
    "MICR": "Magnetic Ink Character Recognition"
}

print("Indian Financial Terminology:")
for term, definition in indian_financial_terms.items():
    print(f"• {term}: {definition}")

---

## 2. Core Prompt Engineering Techniques

Let's explore different prompt engineering techniques using Indian financial transaction examples.

### 2.1 Zero-Shot Prompting: UPI Transaction Classification

In [ ]:
# Zero-shot UPI transaction classification
upi_classification_template = ChatPromptTemplate.from_messages([
    ("system", """You are an Indian financial transaction analyzer. Classify UPI transactions into these categories:
    - FOOD_DELIVERY (Zomato, Swiggy, restaurants)
    - UTILITIES (electricity, water, gas, mobile recharge)
    - ECOMMERCE (Amazon, Flipkart, online shopping)
    - TRANSPORT (Ola, Uber, petrol, metro)
    - HEALTHCARE (hospitals, medicines, insurance)
    - EDUCATION (school fees, courses, books)
    - INVESTMENT (mutual funds, SIP, stocks)
    - TRANSFER (P2P transfers to friends/family)
    - ENTERTAINMENT (movies, OTT, games)
    - OTHER
    
    Respond with: CATEGORY - Brief reason"""),
    ("human", "UPI Transaction: ₹{amount} to {recipient} | {description}")
])

# Sample Indian UPI transactions
upi_transactions = [
    {"amount": "450", "recipient": "ZOMATO", "description": "Food order from McDonald's"},
    {"amount": "2500", "recipient": "hdfc.payzapp", "description": "Electricity bill payment BESCOM"},
    {"amount": "1200", "recipient": "amazonpay", "description": "Online shopping - Electronics"},
    {"amount": "350", "recipient": "olacabs", "description": "Cab ride to Airport"},
    {"amount": "5000", "recipient": "ramesh.kumar@oksbi", "description": "Birthday gift money"},
    {"amount": "3000", "recipient": "icicipruamc", "description": "SIP investment - ELSS fund"}
]

classification_chain = upi_classification_template | llm

print("=== ZERO-SHOT UPI TRANSACTION CLASSIFICATION ===")
for i, transaction in enumerate(upi_transactions, 1):
    response = classification_chain.invoke(transaction)
    print(f"{i}. ₹{transaction['amount']} → {transaction['recipient']}")
    print(f"   Description: {transaction['description']}")
    print(f"   → {response.content}\n")

### 2.2 One-Shot Prompting: IFSC Code Validation

In [ ]:
# One-shot IFSC code validation and information extraction
ifsc_validation_template = ChatPromptTemplate.from_messages([
    ("system", "You validate and provide information about Indian IFSC codes. Follow the example format."),
    ("human", "IFSC Code: SBIN0000123"),
    ("ai", """✅ **VALID IFSC CODE**

**Bank Details:**
- Bank: State Bank of India (SBI)
- Branch Code: 0000123
- Format: SBIN (Bank) + 0 (Reserved) + 000123 (Branch)

**Status:** Active and operational
**Supports:** NEFT, RTGS, IMPS
**Bank Type:** Public Sector Bank"""),
    ("human", "IFSC Code: {ifsc_code}")
])

# Test with various IFSC codes
test_ifsc_codes = [
    "HDFC0001234",  # HDFC Bank
    "ICIC0000456",  # ICICI Bank
    "UTIB0000789",  # Axis Bank (formerly UTI Bank)
    "YESB0000321"   # Yes Bank
]

ifsc_chain = ifsc_validation_template | llm

print("=== ONE-SHOT IFSC CODE VALIDATION ===")
for i, ifsc in enumerate(test_ifsc_codes, 1):
    response = ifsc_chain.invoke({"ifsc_code": ifsc})
    print(f"Test {i}: {ifsc}")
    print(response.content)
    print("\n" + "="*50 + "\n")

### 2.3 Few-Shot Prompting: Indian Financial Document Analysis

In [ ]:
# Few-shot learning for Indian financial document classification
doc_examples = [
    {
        "document": "Account Number: 123456789012, IFSC: SBIN0001234, Balance: ₹45,670, Last Transaction: UPI Credit ₹2,500",
        "classification": "BANK_STATEMENT - Savings account statement with UPI transaction history"
    },
    {
        "document": "PAN: ABCDE1234F, GST Number: 12ABCDE1234F1Z5, Invoice Amount: ₹1,18,000 (including GST ₹18,000)",
        "classification": "GST_INVOICE - Business invoice with tax details and PAN information"
    },
    {
        "document": "Policy Number: LIC123456789, Premium: ₹50,000/year, Maturity Amount: ₹15,00,000, Term: 20 years",
        "classification": "INSURANCE_POLICY - Life insurance policy document with premium and maturity details"
    }
]

# Create few-shot template for document classification
doc_example_prompt = ChatPromptTemplate.from_messages([
    ("human", "Document: {document}"),
    ("ai", "Classification: {classification}")
])

few_shot_doc_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=doc_example_prompt,
    examples=doc_examples
)

final_doc_prompt = ChatPromptTemplate.from_messages([
    ("system", """You classify Indian financial documents into categories:
    BANK_STATEMENT, GST_INVOICE, INSURANCE_POLICY, LOAN_DOCUMENT, 
    INVESTMENT_STATEMENT, TAX_DOCUMENT, SALARY_SLIP, OTHER
    
    Provide classification with brief explanation of key identifiers."""),
    few_shot_doc_prompt,
    ("human", "Document: {document}")
])

# Test documents
test_documents = [
    "Employee ID: E12345, CTC: ₹12,00,000, PF Contribution: ₹14,400, TDS: ₹8,500, Net Salary: ₹85,600",
    "Loan Account: HL123456789, Principal: ₹45,00,000, EMI: ₹42,500, Interest Rate: 8.5%, Tenure: 20 years",
    "Folio Number: MF987654321, NAV: ₹156.78, Units: 1,250.50, Current Value: ₹1,96,078, SIP: ₹10,000/month"
]

doc_classification_chain = final_doc_prompt | llm

print("=== FEW-SHOT INDIAN FINANCIAL DOCUMENT CLASSIFICATION ===")
for i, doc in enumerate(test_documents, 1):
    response = doc_classification_chain.invoke({"document": doc})
    print(f"Document {i}:")
    print(f"Content: {doc[:80]}...")
    print(f"Result: {response.content}\n")

### 2.4 Chain-of-Thought: Indian Tax Calculation

In [ ]:
# Chain-of-thought for Indian income tax calculation
tax_calculation_template = ChatPromptTemplate.from_messages([
    ("system", """You are an Indian tax consultant. Calculate income tax step by step using current tax slabs.
    
    For FY 2023-24, New Tax Regime slabs (₹):
    - 0 to 3,00,000: 0%
    - 3,00,001 to 6,00,000: 5%
    - 6,00,001 to 9,00,000: 10%
    - 9,00,001 to 12,00,000: 15%
    - 12,00,001 to 15,00,000: 20%
    - Above 15,00,000: 30%
    
    Also consider:
    - Health and Education Cess: 4% on total tax
    - Standard deduction: ₹50,000 (if applicable)
    
    Show step-by-step calculation."""),
    ("human", """Calculate income tax for:
    
    Annual Income: ₹{annual_income}
    Regime: {tax_regime}
    Deductions: {deductions}
    
    Please calculate step by step:
    1. Gross Income
    2. Applicable Deductions
    3. Taxable Income
    4. Tax Calculation (slab-wise)
    5. Health & Education Cess
    6. Total Tax Liability
    7. Effective Tax Rate""")
])

# Test case: Middle-class Indian taxpayer
taxpayer_details = {
    "annual_income": "14,50,000",
    "tax_regime": "New Tax Regime",
    "deductions": "Standard Deduction ₹50,000, No other deductions in new regime"
}

tax_chain = tax_calculation_template | llm
response = tax_chain.invoke(taxpayer_details)

print("=== CHAIN-OF-THOUGHT: INDIAN INCOME TAX CALCULATION ===")
print(response.content)

---

## 3. Tree of Thoughts Methodology

Tree of Thoughts is an advanced reasoning technique that explores multiple reasoning paths simultaneously, perfect for complex financial decision-making scenarios.

### 3.1 Tree of Thoughts: Indian Investment Decision Making

In [ ]:
# Tree of Thoughts template for investment decision making
tot_investment_template = ChatPromptTemplate.from_messages([
    ("system", """You are using Tree of Thoughts methodology for Indian investment decisions.
    
    Process:
    1. **Generate Multiple Thought Branches** - Consider 3 different investment approaches
    2. **Evaluate Each Branch** - Analyze pros/cons, risks, returns for Indian market
    3. **Prune Weak Branches** - Eliminate unsuitable options
    4. **Expand Promising Branches** - Develop detailed strategies
    5. **Final Decision** - Choose optimal path with reasoning
    
    Consider Indian market specifics:
    - Tax implications (LTCG, STCG)
    - Inflation impact
    - Regulatory environment
    - Currency risks for international investments
    
    Format your response with clear tree structure using emojis and sections."""),
    ("human", """Investment Scenario:
    
    Profile: {profile}
    Amount: {amount}
    Timeline: {timeline}
    Goal: {goal}
    Risk Tolerance: {risk_tolerance}
    
    Use Tree of Thoughts to recommend the best investment strategy.""")
])

# Complex investment scenario
investment_scenario = {
    "profile": "32-year-old software engineer in Bangalore, ₹15L annual income, married with 1 child",
    "amount": "₹10,00,000 lump sum + ₹25,000/month SIP capacity",
    "timeline": "15 years until child's higher education",
    "goal": "₹50,00,000 corpus for child's education (domestic or international)",
    "risk_tolerance": "Moderate to high (can handle 20-30% volatility)"
}

tot_chain = tot_investment_template | llm
response = tot_chain.invoke(investment_scenario)

print("=== TREE OF THOUGHTS: INDIAN INVESTMENT DECISION MAKING ===")
print(response.content)

### 3.2 Tree of Thoughts: Home Loan Decision Analysis

In [ ]:
# Tree of Thoughts for home loan decision in Indian context
tot_homeloan_template = ChatPromptTemplate.from_messages([
    ("system", """Using Tree of Thoughts methodology, analyze Indian home loan decisions.
    
    Consider multiple branches:
    🏠 **Branch 1: Buy Now** - Current market conditions, loan terms, EMI impact
    ⏳ **Branch 2: Wait & Save** - Build larger down payment, market timing
    🏘️ **Branch 3: Alternative Options** - Rent vs buy, different locations/property types
    
    Evaluate each branch considering:
    - EMI-to-income ratio (recommended <40%)
    - Indian tax benefits (Section 80C, 24B)
    - Property registration costs (stamp duty, registration)
    - Interest rate trends and RBI policy
    - Real estate market cycles in specific city
    - Liquidity impact on overall finances
    
    Provide decision tree with clear reasoning at each node."""),
    ("human", """Home Purchase Decision:
    
    Personal Details: {personal_details}
    Financial Position: {financial_position}
    Property Details: {property_details}
    Current Market: {market_conditions}
    
    Analyze using Tree of Thoughts methodology.""")
])

# Home loan decision scenario
homeloan_scenario = {
    "personal_details": "28-year-old marketing manager in Mumbai, married, planning family in 2-3 years",
    "financial_position": "Monthly income: ₹1,20,000, Savings: ₹25,00,000, Current rent: ₹35,000, Other EMIs: ₹15,000",
    "property_details": "2BHK in Thane, Price: ₹85,00,000, Registration: ₹6,00,000, Loan required: ₹70,00,000",
    "market_conditions": "Interest rates: 8.5-9%, Property prices stable, High rental yields in area"
}

homeloan_chain = tot_homeloan_template | llm
response = homeloan_chain.invoke(homeloan_scenario)

print("=== TREE OF THOUGHTS: HOME LOAN DECISION ANALYSIS ===")
print(response.content)

---

## 4. Indian Financial Use Cases

Real-world applications of prompt engineering in Indian financial services.

### 4.1 UPI Fraud Detection System

In [ ]:
# UPI fraud detection with Indian context
upi_fraud_detection_template = ChatPromptTemplate.from_messages([
    ("system", """You are an advanced UPI fraud detection system for Indian banking.
    
    Analyze transactions for fraud indicators:
    
    🚨 **HIGH RISK Indicators:**
    - Multiple high-value transactions in short time
    - Transactions to new/unverified VPAs
    - Unusual timing (2 AM - 6 AM)
    - Geographic inconsistency (location jumps)
    - Round number amounts near daily limits
    - Multiple failed attempts followed by success
    
    ⚠️ **MEDIUM RISK Indicators:**
    - First-time large transactions
    - Cross-bank transfers to unknown accounts
    - Transactions during festival/holiday periods
    
    ✅ **LOW RISK Indicators:**
    - Regular merchant payments
    - Known contact transactions
    - Consistent behavioral patterns
    
    Provide Risk Score (0-100) and recommended action."""),
    ("human", """Analyze UPI Transaction Pattern:
    
    User Profile: {user_profile}
    Transaction History: {transaction_history}
    Current Transaction: {current_transaction}
    Device/Location: {device_info}
    
    Provide fraud risk assessment.""")
])

# Suspicious transaction scenario
suspicious_scenario = {
    "user_profile": "Account age: 2 years, Typical monthly volume: ₹50,000, Location: Delhi, Occupation: Teacher",
    "transaction_history": """Last 24 hours:
    - 02:15 AM: ₹49,500 to unknown.user@paytm (FAILED - insufficient balance)
    - 02:18 AM: ₹49,500 to unknown.user@paytm (FAILED - daily limit)
    - 02:22 AM: ₹45,000 to unknown.user@paytm (SUCCESS)
    - 02:25 AM: ₹25,000 to different.person@phonepe (SUCCESS)
    - 02:30 AM: ₹30,000 to third.account@googlepay (PENDING)""",
    "current_transaction": "₹50,000 to suspicious.account@upi - P2P transfer with note: 'urgent family emergency'",
    "device_info": "New device login from Mumbai (600km from usual location), IP address change"
}

fraud_chain = upi_fraud_detection_template | llm
response = fraud_chain.invoke(suspicious_scenario)

print("=== UPI FRAUD DETECTION ANALYSIS ===")
print(response.content)

### 4.2 Indian Mutual Fund SIP Recommendation Engine

In [ ]:
# Mutual fund SIP recommendation system
sip_recommendation_template = ChatPromptTemplate.from_messages([
    ("system", """You are a certified Indian mutual fund advisor providing SIP recommendations.
    
    Consider these factors:
    📊 **Fund Categories:**
    - Large Cap: Stable, lower risk (Nifty 50, Sensex funds)
    - Mid Cap: Moderate risk, higher growth potential
    - Small Cap: High risk, highest growth potential
    - ELSS: Tax saving with 3-year lock-in
    - Hybrid: Debt + Equity mix
    - Index Funds: Low cost, market returns
    
    💰 **Tax Implications:**
    - ELSS: 80C deduction up to ₹1.5L
    - LTCG: >₹1L gains taxed at 10% (after 1 year)
    - STCG: 15% tax (within 1 year)
    
    📈 **Asset Allocation by Age:**
    - 20s-30s: 70-80% equity, 20-30% debt
    - 40s: 60-70% equity, 30-40% debt
    - 50s+: 40-50% equity, 50-60% debt
    
    Recommend specific fund houses and schemes available in Indian market."""),
    ("human", """SIP Investment Planning:
    
    Investor Profile: {investor_profile}
    Financial Goals: {financial_goals}
    Investment Capacity: {investment_capacity}
    Risk Profile: {risk_profile}
    
    Provide detailed SIP recommendations with fund suggestions.""")
])

# SIP investment scenario
sip_scenario = {
    "investor_profile": "26-year-old IT professional in Pune, ₹12L annual income, single, no dependents",
    "financial_goals": "Emergency fund: ₹5L (6 months), House down payment: ₹20L (7 years), Retirement: ₹2Cr (35 years)",
    "investment_capacity": "₹30,000/month total (₹15,000 for equity, ₹10,000 for debt, ₹5,000 tax saving)",
    "risk_profile": "High risk tolerance, 10+ year investment horizon, comfortable with 30-40% volatility"
}

sip_chain = sip_recommendation_template | llm
response = sip_chain.invoke(sip_scenario)

print("=== INDIAN MUTUAL FUND SIP RECOMMENDATIONS ===")
print(response.content)

### 4.3 GST Compliance and Invoice Analysis

In [ ]:
# GST compliance checker for Indian businesses
gst_compliance_template = ChatPromptTemplate.from_messages([
    ("system", """You are a GST compliance expert for Indian businesses.
    
    Check invoices for:
    ✅ **Mandatory Fields:**
    - Invoice number and date
    - GSTIN of supplier and recipient
    - Description of goods/services
    - HSN/SAC codes
    - Taxable value and GST amount
    - Place of supply
    
    📊 **GST Rates (2024):**
    - 0%: Essential items (rice, wheat, milk)
    - 5%: Household items (tea, coffee, medicines)
    - 12%: Processed foods, mobile phones
    - 18%: Most goods and services (IT, financial services)
    - 28%: Luxury items (cars, tobacco, air conditioners)
    
    🔍 **Compliance Checks:**
    - Input Tax Credit (ITC) eligibility
    - Reverse charge mechanism applicability
    - Interstate vs intrastate transaction rules
    
    Identify compliance issues and suggest corrections."""),
    ("human", """GST Invoice Analysis:
    
    Invoice Details: {invoice_details}
    Business Type: {business_type}
    Transaction Type: {transaction_type}
    
    Check for GST compliance and identify any issues.""")
])

# GST invoice for analysis
gst_scenario = {
    "invoice_details": """Invoice No: INV/2024/001
    Date: 15-March-2024
    Supplier GSTIN: 29ABCDE1234F1Z5 (Karnataka)
    Recipient GSTIN: 27FGHIJ5678K2L9 (Maharashtra)
    Items: Software Development Services (HSN: 998314)
    Taxable Value: ₹1,00,000
    CGST: Not mentioned
    SGST: Not mentioned  
    IGST: ₹18,000 (18%)
    Total: ₹1,18,000
    Place of Supply: Maharashtra""",
    "business_type": "IT Services company providing software development",
    "transaction_type": "B2B interstate service transaction"
}

gst_chain = gst_compliance_template | llm
response = gst_chain.invoke(gst_scenario)

print("=== GST COMPLIANCE ANALYSIS ===")
print(response.content)

---

## 5. Advanced Techniques & Role-Based Prompting

Specialized roles for different aspects of Indian financial services.

### 5.1 Role-Based Prompting: Financial Advisor Personas

In [ ]:
# Different financial advisor personas for Indian context
advisor_personas = {
    "rural_banking_specialist": {
        "role": "You are a rural banking specialist working with farmers and small business owners in Indian villages.",
        "expertise": "Agricultural loans, crop insurance, government schemes (PM-KISAN, PMFBY), financial literacy in regional languages",
        "communication": "Simple language, practical examples, culturally sensitive, patient explanations"
    },
    "urban_investment_advisor": {
        "role": "You are a premium investment advisor for high-net-worth Indians in metro cities.",
        "expertise": "Portfolio management, tax optimization, international investments, alternative investments (REITs, AIFs)",
        "communication": "Sophisticated analysis, data-driven insights, global market perspective"
    },
    "middle_class_planner": {
        "role": "You are a financial planner specializing in Indian middle-class families.",
        "expertise": "Goal-based planning, insurance, children's education, retirement planning, tax-saving investments",
        "communication": "Practical advice, cost-effective solutions, emphasis on security and growth"
    }
}

def create_advisor_template(persona_key):
    persona = advisor_personas[persona_key]
    return ChatPromptTemplate.from_messages([
        ("system", f"""{persona['role']}
        
        Your expertise includes: {persona['expertise']}
        
        Communication style: {persona['communication']}
        
        Always consider Indian market conditions, regulations, and cultural factors."""),
        ("human", "{query}")
    ])

# Test query for each persona
test_query = "I have ₹2 lakh saved and want to invest for my daughter's education in 10 years. What should I do?"

print("=== ROLE-BASED FINANCIAL ADVICE COMPARISON ===")
print(f"Query: {test_query}\n")

for persona_name, persona_data in advisor_personas.items():
    template = create_advisor_template(persona_name)
    chain = template | llm
    response = chain.invoke({"query": test_query})
    
    print(f"=== {persona_name.replace('_', ' ').title()} ===")
    print(response.content[:400] + "...\n")

### 5.2 Regional Language Support

In [ ]:
# Multi-language financial advisory template
multilingual_template = ChatPromptTemplate.from_messages([
    ("system", """You provide financial advice in Indian regional languages.
    
    When explaining financial concepts:
    1. Use simple, everyday language
    2. Include English terms in brackets for technical words
    3. Use local examples and cultural references
    4. Explain concepts step-by-step
    
    Language Guidelines:
    - Hindi: Use Devanagari numerals where appropriate
    - Tamil: Include cultural context relevant to Tamil Nadu
    - Bengali: Reference local banking practices in West Bengal
    
    Always maintain accuracy of financial information regardless of language."""),
    ("human", """Language: {language}
    Question: {question}
    
    Please provide financial advice in the requested language.""")
])

# Test multilingual financial advice
multilingual_queries = [
    {
        "language": "Hindi",
        "question": "Mujhe apne bachche ki padhai ke liye paisa jama karna hai. SIP kya hota hai?"
    },
    {
        "language": "Tamil",
        "question": "What is the best way to save money for buying a house in Chennai?"
    },
    {
        "language": "Bengali", 
        "question": "Amar retirement er jonno koto taka save korte hobe?"
    }
]

multilingual_chain = multilingual_template | llm

print("=== MULTILINGUAL FINANCIAL ADVISORY ===")
for i, query in enumerate(multilingual_queries, 1):
    response = multilingual_chain.invoke(query)
    print(f"Query {i} ({query['language']}):")
    print(f"Question: {query['question']}")
    print(f"Response: {response.content[:300]}...\n")

---

## 6. Practical Indian Financial Scenarios

Complex, real-world scenarios combining multiple prompt engineering techniques.

### 6.1 Comprehensive Financial Health Check

In [ ]:
# Comprehensive financial health assessment for Indians
financial_health_template = ChatPromptTemplate.from_messages([
    ("system", """You are conducting a comprehensive financial health check for an Indian family.
    
    Analyze across these dimensions:
    
    💰 **INCOME & EXPENSES (Score: 0-25)**
    - Income stability and growth potential
    - Expense management and savings rate
    - Debt-to-income ratio
    
    🛡️ **PROTECTION (Score: 0-25)**
    - Life insurance coverage (10-12x annual income)
    - Health insurance adequacy
    - Emergency fund (6-12 months expenses)
    
    📈 **INVESTMENTS (Score: 0-25)**
    - Asset allocation appropriateness
    - Diversification across asset classes
    - Tax-efficient investing (ELSS, PPF, etc.)
    
    🎯 **GOALS & PLANNING (Score: 0-25)**
    - Clear financial goals with timelines
    - Retirement planning adequacy
    - Children's education planning
    
    Provide:
    1. Overall Financial Health Score (0-100)
    2. Category-wise analysis
    3. Top 3 priority actions
    4. Long-term roadmap
    
    Consider Indian context: tax laws, inflation, typical life goals."""),
    ("human", """Financial Profile:
    
    Personal: {personal_details}
    Income: {income_details}
    Expenses: {expense_details}
    Assets: {asset_details}
    Liabilities: {liability_details}
    Insurance: {insurance_details}
    Goals: {goals}
    
    Please conduct comprehensive financial health assessment.""")
])

# Detailed financial profile
family_profile = {
    "personal_details": "Raj (35) & Priya (32), IT professionals in Hyderabad, 2 children (8 & 5 years)",
    "income_details": "Combined income: ₹28L/year (Raj: ₹18L, Priya: ₹10L), Rental income: ₹25K/month",
    "expense_details": "Monthly expenses: ₹1.2L (rent ₹35K, education ₹20K, household ₹40K, EMIs ₹25K)",
    "asset_details": "Savings: ₹12L, FDs: ₹8L, MF/Stocks: ₹15L, EPF: ₹18L, Property: ₹45L (rental)",
    "liability_details": "Home loan: ₹25L remaining (₹22K EMI), Car loan: ₹3L (₹8K EMI)",
    "insurance_details": "Life: ₹50L term (Raj), ₹25L term (Priya), Health: ₹10L family floater",
    "goals": "Children education: ₹50L each (10-13 years), Retirement: ₹3Cr (25 years), Second home: ₹80L (5 years)"
}

health_check_chain = financial_health_template | llm
response = health_check_chain.invoke(family_profile)

print("=== COMPREHENSIVE FINANCIAL HEALTH CHECK ===")
print(response.content)

### 6.2 Agricultural Loan Assessment (Tree of Thoughts)

In [ ]:
# Agricultural loan assessment using Tree of Thoughts
agri_loan_tot_template = ChatPromptTemplate.from_messages([
    ("system", """You are an agricultural loan officer using Tree of Thoughts methodology.
    
    Evaluate loan applications through multiple reasoning branches:
    
    🌱 **Branch 1: Crop Viability Analysis**
    - Soil quality and water availability
    - Crop selection and market demand
    - Seasonal risks and weather patterns
    - Expected yield and revenue projections
    
    👨‍🌾 **Branch 2: Farmer Profile Assessment**
    - Experience and farming knowledge
    - Land ownership status
    - Previous loan repayment history
    - Family support and alternative income
    
    🏦 **Branch 3: Financial & Risk Analysis**
    - Loan amount vs land value ratio
    - Repayment capacity assessment
    - Crop insurance coverage
    - Government scheme benefits (PM-KISAN, etc.)
    
    Consider:
    - Monsoon dependency and climate risks
    - Market price volatility
    - Government policies and MSP
    - Rural banking challenges
    
    Provide decision tree with risk mitigation strategies."""),
    ("human", """Agricultural Loan Application:
    
    Farmer Profile: {farmer_profile}
    Land Details: {land_details}
    Loan Purpose: {loan_purpose}
    Financial History: {financial_history}
    Regional Context: {regional_context}
    
    Assess loan viability using Tree of Thoughts methodology.""")
])

# Agricultural loan scenario
agri_loan_scenario = {
    "farmer_profile": "Suresh Kumar, age 42, farming for 15 years in Nashik district, Maharashtra. Literate, progressive farmer, active in farmer producer organization",
    "land_details": "10 acres owned land, fertile black cotton soil, drip irrigation system, bore well with good water table",
    "loan_purpose": "₹8,00,000 loan for grape cultivation - trellising, drip system expansion, organic certification, and working capital for 2 years",
    "financial_history": "Previous KCC loan ₹3L (fully repaid), Bank account with 5-year history, PM-KISAN beneficiary, Annual income ₹4-6L from farming",
    "regional_context": "Nashik - major grape growing region, good market access, contract farming opportunities with wineries, climate change impacts on rainfall pattern"
}

agri_loan_chain = agri_loan_tot_template | llm
response = agri_loan_chain.invoke(agri_loan_scenario)

print("=== AGRICULTURAL LOAN ASSESSMENT (TREE OF THOUGHTS) ===")
print(response.content)

---

## 7. Performance Optimization & Validation

Tools and techniques for optimizing prompt performance in Indian financial contexts.

### 7.1 Indian Financial Data Validation

In [ ]:
# Validation template for Indian financial data
validation_template = ChatPromptTemplate.from_messages([
    ("system", """You validate Indian financial data for accuracy and compliance.
    
    Validation Rules:
    
    🔢 **PAN Validation:**
    - Format: ABCDE1234F (5 letters + 4 numbers + 1 letter)
    - 4th character: P for individual, C for company
    
    🏦 **IFSC Validation:**
    - Format: ABCD0123456 (4 letters + 1 zero + 6 characters)
    - First 4 letters: Bank code
    
    📱 **UPI VPA Validation:**
    - Format: username@bankname
    - Valid bank handles: @oksbi, @paytm, @ybl, @axl, etc.
    
    💳 **Account Number:**
    - Length: 9-18 digits (varies by bank)
    - Check digit validation where applicable
    
    🎯 **GST Number:**
    - Format: 15 characters (2 state + 10 PAN + 1 entity + 1 check + 1 default)
    
    For each field, respond:
    ✅ VALID - [explanation]
    ❌ INVALID - [issue and correction needed]
    ⚠️ SUSPICIOUS - [potential issue to verify]"""),
    ("human", """Validate Financial Data:
    
    {financial_data}
    
    Check each field for format correctness and flag any issues.""")
])

# Test financial data with mixed valid/invalid entries
test_financial_data = """
Customer Details:
PAN: ABCDE1234F
GSTIN: 29ABCDE1234F1Z5
Bank Account: 123456789012345
IFSC: SBIN0001234
UPI VPA: john.doe@sbi
Mobile: +91-9876543210

Transaction Details:
Amount: ₹1,25,000
Transaction ID: UPI20240315123456
Reference: Payment for software services
"""

validation_chain = validation_template | llm
response = validation_chain.invoke({"financial_data": test_financial_data})

print("=== INDIAN FINANCIAL DATA VALIDATION ===")
print(response.content)

### 7.2 Prompt Performance Benchmarking

In [ ]:
# Performance benchmarking for Indian financial prompts
import time
from datetime import datetime

class IndianFinancialPromptBenchmark:
    def __init__(self, llm):
        self.llm = llm
        self.results = []
    
    def benchmark_prompt(self, template, test_cases, prompt_name):
        """Benchmark a prompt template with multiple test cases"""
        chain = template | self.llm
        results = {
            'prompt_name': prompt_name,
            'test_count': len(test_cases),
            'responses': [],
            'avg_time': 0,
            'success_rate': 0,
            'accuracy_scores': []
        }
        
        total_time = 0
        successful_responses = 0
        
        for i, test_case in enumerate(test_cases):
            try:
                start_time = time.time()
                response = chain.invoke(test_case['input'])
                end_time = time.time()
                
                response_time = end_time - start_time
                total_time += response_time
                
                # Simple accuracy check (contains expected keywords)
                accuracy = self._check_accuracy(response.content, test_case.get('expected_keywords', []))
                results['accuracy_scores'].append(accuracy)
                
                results['responses'].append({
                    'test_case': i + 1,
                    'response_time': response_time,
                    'response_length': len(response.content),
                    'accuracy': accuracy,
                    'response_preview': response.content[:100] + '...'
                })
                
                successful_responses += 1
                
            except Exception as e:
                results['responses'].append({
                    'test_case': i + 1,
                    'error': str(e),
                    'response_time': 0,
                    'accuracy': 0
                })
        
        results['avg_time'] = total_time / len(test_cases) if test_cases else 0
        results['success_rate'] = (successful_responses / len(test_cases)) * 100 if test_cases else 0
        results['avg_accuracy'] = sum(results['accuracy_scores']) / len(results['accuracy_scores']) if results['accuracy_scores'] else 0
        
        self.results.append(results)
        return results
    
    def _check_accuracy(self, response, expected_keywords):
        """Simple accuracy check based on keyword presence"""
        if not expected_keywords:
            return 100  # No expectations, assume good
        
        response_lower = response.lower()
        found_keywords = sum(1 for keyword in expected_keywords if keyword.lower() in response_lower)
        return (found_keywords / len(expected_keywords)) * 100
    
    def print_benchmark_report(self):
        """Print comprehensive benchmark report"""
        print("=== INDIAN FINANCIAL PROMPT BENCHMARK REPORT ===")
        print(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        
        for result in self.results:
            print(f"📊 **{result['prompt_name']}**")
            print(f"   Test Cases: {result['test_count']}")
            print(f"   Success Rate: {result['success_rate']:.1f}%")
            print(f"   Avg Response Time: {result['avg_time']:.2f}s")
            print(f"   Avg Accuracy: {result['avg_accuracy']:.1f}%")
            print(f"   Avg Response Length: {sum(r.get('response_length', 0) for r in result['responses']) / result['test_count']:.0f} chars")
            print()

# Test benchmark with UPI classification prompt
benchmark = IndianFinancialPromptBenchmark(llm)

# Simple UPI classification template for testing
simple_upi_template = ChatPromptTemplate.from_messages([
    ("system", "Classify UPI transactions as FOOD, TRANSPORT, SHOPPING, UTILITIES, or OTHER"),
    ("human", "Transaction: ₹{amount} to {recipient}")
])

# Test cases for benchmarking
upi_test_cases = [
    {
        'input': {'amount': '450', 'recipient': 'ZOMATO'},
        'expected_keywords': ['FOOD']
    },
    {
        'input': {'amount': '350', 'recipient': 'olacabs'},
        'expected_keywords': ['TRANSPORT']
    },
    {
        'input': {'amount': '1200', 'recipient': 'amazonpay'},
        'expected_keywords': ['SHOPPING']
    },
    {
        'input': {'amount': '2500', 'recipient': 'electricity_board'},
        'expected_keywords': ['UTILITIES']
    }
]

# Run benchmark
benchmark.benchmark_prompt(simple_upi_template, upi_test_cases, "UPI Transaction Classification")
benchmark.print_benchmark_report()

### 7.3 Security and Compliance Checker

In [ ]:
# Security and compliance checker for Indian financial prompts
security_compliance_template = ChatPromptTemplate.from_messages([
    ("system", """You are a security and compliance auditor for Indian financial AI systems.
    
    Check for:
    
    🔒 **Data Privacy (RBI Guidelines):**
    - No storage of sensitive personal data
    - Proper masking of account numbers, PAN, Aadhaar
    - Compliance with data localization requirements
    
    🛡️ **Security Issues:**
    - Potential prompt injection attacks
    - Information leakage in responses
    - Authentication bypass attempts
    
    📋 **Regulatory Compliance:**
    - RBI operational guidelines
    - KYC/AML compliance
    - Consumer protection measures
    - Fair practice codes
    
    🎯 **Response Quality:**
    - Accuracy of financial calculations
    - Appropriate disclaimers
    - Risk warnings where needed
    
    For each check, provide:
    ✅ COMPLIANT
    ⚠️ WARNING - [issue and recommendation]
    ❌ VIOLATION - [critical issue requiring immediate attention]"""),
    ("human", """Security & Compliance Audit:
    
    Prompt Template: {prompt_template}
    Sample Input: {sample_input}
    Sample Output: {sample_output}
    
    Conduct comprehensive security and compliance review.""")
])

# Example prompt and response for audit
audit_scenario = {
    "prompt_template": """System: You provide loan eligibility assessment for Indian customers.
    Human: Customer details - Name: {name}, PAN: {pan}, Income: {income}, CIBIL: {cibil_score}
    Check loan eligibility and provide recommendation.""",
    "sample_input": "Name: Rahul Sharma, PAN: ABCDE1234F, Income: ₹8,50,000, CIBIL: 750",
    "sample_output": """Based on your profile:
    - Excellent CIBIL score of 750
    - Annual income ₹8.5L qualifies for home loan up to ₹60L
    - Recommended EMI: ₹45,000 (max 40% of income)
    - Interest rate: 8.5-9% (subject to bank approval)
    
    Disclaimer: This is indicative assessment only. Final approval subject to bank verification."""
}

security_chain = security_compliance_template | llm
response = security_chain.invoke(audit_scenario)

print("=== SECURITY & COMPLIANCE AUDIT REPORT ===")
print(response.content)

---

## Summary and Best Practices

### 🎯 **Key Takeaways for Indian Financial Prompt Engineering:**

#### **1. Context is King**
- Always include Indian regulatory context (RBI, SEBI, IRDAI)
- Consider local market conditions and cultural factors
- Use appropriate Indian financial terminology

#### **2. Prompt Engineering Techniques Ranking for Indian Finance:**

1. **Tree of Thoughts** - Best for complex decisions (investment, loans)
2. **Chain-of-Thought** - Excellent for calculations and step-by-step analysis
3. **Few-Shot** - Perfect for classification and document processing
4. **Role-Based** - Essential for specialized advice (rural banking, HNI services)
5. **Zero-Shot** - Good for simple, well-defined tasks

#### **3. Indian Financial Domain Specifics:**

**✅ Always Include:**
- Tax implications (current Indian tax slabs)
- Inflation adjustment for long-term planning
- Regulatory compliance requirements
- Cultural sensitivity (joint families, festival expenses)
- Regional variations (state-specific policies)

**❌ Common Pitfalls:**
- Using outdated tax rates or policies
- Ignoring Indian market volatility patterns
- Not considering monsoon impact on agricultural loans
- Overlooking state-specific regulations

#### **4. Security & Compliance:**
- Never store or log sensitive financial data
- Always include appropriate disclaimers
- Follow RBI data localization guidelines
- Implement proper data masking for PII

#### **5. Performance Optimization:**
- Use template caching for frequently used prompts
- Implement validation layers for critical calculations
- Monitor accuracy with domain expert reviews
- Regular updates for changing regulations

### 🚀 **Next Steps:**

1. **Implement A/B Testing** - Compare prompt variations for your use cases
2. **Build Domain Knowledge Base** - Create Indian financial fact database
3. **User Feedback Loop** - Collect and analyze user satisfaction
4. **Regulatory Updates** - Set up alerts for policy changes
5. **Multi-language Expansion** - Scale to more Indian languages

### 📚 **Resources for Continued Learning:**

- **RBI Guidelines**: [www.rbi.org.in](https://www.rbi.org.in)
- **Tax Laws**: Income Tax Department notifications
- **Market Data**: NSE, BSE official sources
- **Rural Finance**: NABARD guidelines and schemes

---

**Remember**: Indian financial services require a unique blend of technical expertise, regulatory knowledge, and cultural sensitivity. The Tree of Thoughts methodology is particularly powerful for complex financial decision-making scenarios common in Indian households and businesses.

Happy prompt engineering for Indian FinTech! 🇮🇳💰🚀